In [ ]:
!pip install pandas transformers torch pipline

In [ ]:
import pandas as pd
from transformers import pipeline
import random

class SimpleEmotionRecommender:
    def __init__(self, csv_path):
        # CSV 데이터 로드
        self.df = pd.read_csv(csv_path)

        # 감정 분석 모델 로드 (한국어 감정 분석에 특화된 모델)
        self.classifier = pipeline(
            "text-classification",
            model="snunlp/KR-FinBert-SC",
            return_all_scores=True
        )

    def analyze_emotion(self, text):
        """텍스트의 감정을 분석하고 확률 점수를 반환"""
        try:
            # 감정 분석 수행
            results = self.classifier(text)[0]

            # 감정 점수 매핑
            emotion_scores = {
                'Sad': 0,
                'Angry': 0,
                'Fear': 0,
                'Happy': 0,
                'Calm': 0
            }

            # 부정적인 표현 키워드
            negative_keywords = {
                'Sad': ['슬프', '우울', '눈물', '아프', '그립', '외롭', '서럽', '괴롭', '울고'],
                'Angry': ['화나', '짜증', '분노', '화가', '빡치', '열받', '싫어', '미워'],
                'Fear': ['무섭', '불안', '걱정', '두렵', '긴장', '조마조마', '떨려']
            }

            # 긍정적인 표현 키워드
            positive_keywords = ['행복', '기쁘', '좋아', '신나', '즐겁', '설레']

            # 텍스트에서 키워드 확인
            for category, keywords in negative_keywords.items():
                for keyword in keywords:
                    if keyword in text:
                        emotion_scores[category] += 1

            for keyword in positive_keywords:
                if keyword in text:
                    emotion_scores['Happy'] += 1

            # 기본 감정 분석 결과 반영
            for result in results:
                if result['label'] == 'negative':
                    score = result['score']
                    if score > 0.6:  # 강한 부정
                        # 키워드가 없으면 Sad가 기본
                        if sum(emotion_scores[e] for e in ['Sad', 'Angry', 'Fear']) == 0:
                            emotion_scores['Sad'] += 1
                elif result['label'] == 'positive':
                    emotion_scores['Happy'] += result['score']
                elif result['label'] == 'neutral':
                    emotion_scores['Calm'] += result['score']

            # 가장 높은 점수의 감정 선택
            selected_emotion = max(emotion_scores.items(), key=lambda x: x[1])
            max_score = max(result['score'] for result in results)

            return selected_emotion[0], max_score

        except Exception as e:
            print(f"감정 분석 중 오류 발생: {str(e)}")
            return 'Sad', 0.5  # 오류 발생시 기본적으로 Sad 반환

    def recommend_songs(self, text, n_songs=3):
        """감정 분석 결과를 바탕으로 노래 추천"""
        # 감정 분석
        emotion, confidence = self.analyze_emotion(text)

        # 감정에 따른 메시지
        emotion_messages = {
            'Happy': "기쁜 마음이 느껴지네요! 이 기분을 더 업시켜줄 노래들을 추천해드려요.",
            'Sad': "많이 우울하시군요. 공감할 수 있는 노래들을 들려드릴게요.",
            'Angry': "화가 나는 상황이신가봐요. 이 노래들이 마음을 달래는데 도움이 되길 바라요.",
            'Fear': "불안한 마음이 드시나봐요. 이 곡들이 위로가 되길 바랍니다.",
            'Calm': "평온한 마음을 즐기실 수 있는 노래들을 추천해드려요."
        }

        # 해당 감정의 노래들 필터링
        emotion_songs = self.df[self.df['tags'] == emotion]

        if len(emotion_songs) == 0:
            return f"죄송합니다. 해당하는 감정의 노래를 찾을 수 없습니다."

        # 랜덤하게 n_songs개 선택
        recommended = emotion_songs.sample(n=min(n_songs, len(emotion_songs)))

        # 결과 포매팅
        result = f"\n{emotion_messages[emotion]}\n"
        result += f"감지된 감정: {emotion} (확률: {confidence:.2f})\n\n추천 노래 목록:\n"
        for i, (_, song) in enumerate(recommended.iterrows(), 1):
            result += f"{i}. {song['title']} - {song['artist']} ({song['release_date']})\n"

        return result

def main():
    # CSV 파일 경로
    csv_path = 'songs.csv'

    print("=== TUYU 감정 기반 음악 추천 시스템 ===")
    print("지금 기분이나 상황을 자유롭게 이야기해주세요.")
    print("종료하려면 '종료'를 입력하세요.\n")

    try:
        print("모델을 로딩중입니다...")
        recommender = SimpleEmotionRecommender(csv_path)
        print("준비완료!")

        while True:
            user_input = input("\n당신: ")

            if user_input.lower() == '종료':
                print("프로그램을 종료합니다.")
                break

            if not user_input:
                print("내용을 입력해주세요!")
                continue

            try:
                result = recommender.recommend_songs(user_input)
                print(result)
            except Exception as e:
                print("죄송합니다. 오류가 발생했습니다. 다시 시도해주세요.")
                print(f"오류 내용: {str(e)}")

    except Exception as e:
        print(f"시스템 초기화 중 오류가 발생했습니다: {str(e)}")

if __name__ == "__main__":
    main()

=== TUYU 감정 기반 음악 추천 시스템 ===
지금 기분이나 상황을 자유롭게 이야기해주세요.
종료하려면 '종료'를 입력하세요.

모델을 로딩중입니다...


Device set to use cpu
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


준비완료!

당신: 나 행복해

기쁜 마음이 느껴지네요! 이 기분을 더 업시켜줄 노래들을 추천해드려요.
감지된 감정: Happy (확률: 1.00)

추천 노래 목록:
1. 太陽になれるかな - TUYU (2020-02-28)
2. くらべられっ子 (TUYU Remix) - TUYU (2020-05-31)
3. 忠犬ハチ - TUYU (2021-07-04)


당신: 나 우울해

많이 우울하시군요. 공감할 수 있는 노래들을 들려드릴게요.
감지된 감정: Sad (확률: 0.60)

추천 노래 목록:
1. やっぱり雨は降るんだね - TUYU (2022-02-19)
2. かくれんぼっち - TUYU (2021-01-31)
3. 過去に囚われている - TUYU (2020-08-28)


당신: 나 슬퍼

평온한 마음을 즐기실 수 있는 노래들을 추천해드려요.
감지된 감정: Calm (확률: 0.87)

추천 노래 목록:
1. 夏浅し[Inst.] - TUYU (2022-02-19)


당신: 종료
프로그램을 종료합니다.
